# B3: Day vao bo nho Redis

In [1]:
import redis
import json
import pandas as pd
import numpy as np
import datetime

def insertRedis(df):
    # Kết nối đến Redis server
    redis_host = 'localhost'  # Thay 'localhost' bằng địa chỉ Redis server của bạn
    redis_port = 6379  # Thay 6379 bằng cổng Redis server của bạn
    r = redis.StrictRedis(host=redis_host, port=redis_port, decode_responses=True)

    df = df.to_frame()  
    # Lặp qua từng cột của DataFrame
    for column_name in df.columns:
        column_value = df[column_name].to_dict()
        for index, value in column_value.items():
            if (index != 'Timestamp' and index != 'Date_time'):
                datetime_string = str(column_value['Date_time'])
                datetime_object = datetime.datetime.strptime(datetime_string, '%Y-%m-%d %H:%M:%S')
                date_string = datetime_object.strftime('%Y-%m-%d %H:%M:%S')
                r.hset("H_CHECK_MA_ATR", index, value)
            elif (index == 'Date_time'):
                datetime_string = str(column_value['Date_time'])
                datetime_object = datetime.datetime.strptime(datetime_string, '%Y-%m-%d %H:%M:%S')
                date_string = datetime_object.strftime('%Y-%m-%d %H:%M:%S')
                r.hset("H_CHECK_MA_ATR", index, date_string)

    # Đóng kết nối đến Redis server
    r.connection_pool.disconnect()

# B1: Lay du lieu va B2: Kiem tra chien luoc

In [2]:
import ccxt
import time 

def calculate_ma(data, period):
    return data['Close'].rolling(window=period).mean()

def calculate_atr(data, period):
    true_range = data['High'] - data['Low']
    atr = true_range.rolling(window=period).mean()
    return atr

def main():
    chaylandau = 0
    symbol = "EUR/USDT"  # Sử dụng cặp tiền tệ BNB/USDT trên sàn Binance EURUSD
    ma_fast_period = 50  # Gia tri cua MA nhanh
    ma_slow_period = 200  # Gia tri cua MA chậm
    atr_period = 14  # Gia tri cua ATR

    # Khởi tạo kết nối đến sàn Binance
    exchange = ccxt.binance()

    while True: # shedule
        # Lấy dữ liệu thị trường từ sàn Binance
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe='15m', limit=1000)  # Lấy 1000 điểm dữ liệu

        # Chuyển dữ liệu thành DataFrame
        df = pd.DataFrame(ohlcv, columns=['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume'])

        df['Date_time'] = pd.to_datetime(df['Timestamp'], unit='ms')

        # Tính toán MA nhanh và MA chậm
        df['MA_Fast'] = calculate_ma(df, ma_fast_period)
        df['MA_Slow'] = calculate_ma(df, ma_slow_period)

        # Tính toán ATR
        df['ATR'] = calculate_atr(df, atr_period)       
        
        # Tính toán TP, SL
        df['TP'] = 0.8 * df['ATR']
        df['SL'] = 0.5 * df['TP']

        # Đợi 10s trước khi lặp lại
        current_time = time.localtime()
        if (current_time.tm_min == 2 or current_time.tm_min == 17 or current_time.tm_min == 32
            or current_time.tm_min == 47):
            df['CurTime'] = current_time.tm_min
             # So sánh MA nhanh và MA chậm, và kiểm tra ATR        
            df.to_csv("CheckMA_ATR.csv", index=True)
            if df['MA_Fast'].iloc[-1] > df['MA_Slow'].iloc[-1] and df['ATR'].iloc[-1] < 10:  # Ví dụ về ngưỡng ATR là 10
                print(df.iloc[-1])
                insertRedis(df.iloc[-1])
                df.iloc[-1].to_csv("CheckMA_ATR1.csv", index=True)
                print("MA Fast > MA Slow and ATR < 10 at current time - Buy Signal")
        else:
            if(chaylandau == 0):
                df['CurTime'] = current_time.tm_min
                df.to_csv("CheckMA_ATR.csv", index=True)
                if df['MA_Fast'].iloc[-1] > df['MA_Slow'].iloc[-1] and df['ATR'].iloc[-1] < 10:  # Ví dụ về ngưỡng ATR là 10
                    print(df.iloc[-1])
                    insertRedis(df.iloc[-1])
                    df.iloc[-1].to_csv("CheckMA_ATR1.csv", index=True)                    
                    print("MA Fast > MA Slow and ATR < 10 at current time - Buy Signal")
            chaylandau = chaylandau + 1
            time.sleep(30)

if __name__ == "__main__":
    main()

Timestamp          1706019300000
Open                      1.0868
High                       1.087
Low                       1.0867
Close                     1.0869
Volume                   51877.2
Date_time    2024-01-23 14:15:00
MA_Fast                 1.088712
MA_Slow                 1.088667
ATR                     0.000457
TP                      0.000366
SL                      0.000183
CurTime                       16
Name: 999, dtype: object
MA Fast > MA Slow and ATR < 10 at current time - Buy Signal
Timestamp          1706019300000
Open                      1.0868
High                       1.087
Low                       1.0867
Close                     1.0869
Volume                   88420.8
Date_time    2024-01-23 14:15:00
MA_Fast                 1.088712
MA_Slow                 1.088667
ATR                     0.000457
TP                      0.000366
SL                      0.000183
CurTime                       17
Name: 999, dtype: object
MA Fast > MA Slow and ATR < 10 a